## 01_py_basics

#### Python repository for general data manipulation techniques and working with pandas. 

## This notebook will cover:

**1. Selecting columns**
* basic selection and drop columns
* select using pattern recognition
* relocate columns

**2. Creating new columns and values**
* basics of creating new columns
* basic sums and numerical column manipulation
* row sums and combine columns together
* conditionally create columns (ifelse / case_when equivlient)

**3. Filtering data**
* basic filtering
* filter NAs
* filter using arrays

**4. Aggregations using groupby**
* basic groupby aggregation
* multiple calculations
* groupby with conditional calculations
* unnest concatonated cells

**5. Joins (merge)**
* left join using merge
* concatonate dfs together

**6. Other operators**
* remove duplicates
* sort data

**Glossary**
Glossary of functions used throughout notebook. 

### 0. Set up ---

Basic set up to load and inspect data before any data exploration and analysis:

First step is to load in basic Python libraris and the data..

**Please note the trade data is not real-world values rather dummy data for the purposes of demonstrations.**

In [ ]:
# pandas and numpy are universally used in python, like tidyverse is in R. 
import pandas as pd
import numpy as np

!pip install openpyxl

# chnage from scientific notation 
pd.set_option('display.float_format', lambda x: '%.5f' % x)

trade = pd.read_excel("data/trade_data.xlsx") # upload xlsxl
tariff = pd.read_excel("data/tariff_data.xlsx")
uk_trqs = pd.read_csv("data/uk_trqs.csv",dtype={'quota__order_number': str})
# upload csv

In [ ]:
trade.head()

basic df exploration:

In [ ]:
# column names and types:
trade.dtypes

In [ ]:
# df summary:
trade.info()

using .info is very useful as in additional to Dtypes being printed you are provided with the "non-null" values or in other words NAs. For example the supression notes column is only NA values.

In [ ]:
# summarise numerical values
trade.describe()

In [ ]:
# simple df dimensions use shape:
trade.shape

**Note:** that the year column is uploaded as a value. It may be preferable to work with a character type rather than value for this column. When uploading data the data type can be specified

In [ ]:
trade2 = pd.read_excel("data/trade_data.xlsx",dtype={'Year': str}) # convert year to string when uploading data
trade3 = pd.read_excel("data/trade_data.xlsx",dtype=str) # all columns as string
trade4 = pd.read_excel("data/trade_data.xlsx",dtype={'Value GBP': np.float64}) # convert value to float opposed to integer. Floats allows for decimal points
print(trade2.dtypes,trade3.dtypes,trade4.dtypes)

In [ ]:
# want float for value so re-upload trade data:
trade = pd.read_excel("data/trade_data.xlsx",dtype={'Value GBP': np.float64})
trade.info()

## janitor - clean_names() equivalent. 

Working with cleaner string/column names is highlgihy recommended. 

In [ ]:
trade.columns = trade.columns.str.lower().str.replace(" ","_")
trade.dtypes

In [ ]:
# using function - helpful if multiple dataframes to convert.
def  cleanCols(df): 
    df.columns = df.columns.str.lower().str.replace(" ","_")
    return(df)

trade = cleanCols(trade)
trade2 = cleanCols(trade2)
trade3 = cleanCols(trade3)
tariff = cleanCols(tariff)

## 1. Select columns ----

basic selection:

In [ ]:
trade2 = trade[["year","flow","commodity_code","country_name","value_gbp"]]
trade2.dtypes

In [ ]:
# use an array:
cols = ["year","flow","commodity_code","country_name","value_gbp"]
trade2 = trade[cols]

In [ ]:
trade2.dtypes

#### Select by column index numbers

In [ ]:
df = trade.copy()
# select first two columns
df2 = df.iloc[:,0:2]
df2.head()

In [ ]:
# select last two columns 
df2 = df.iloc[:,5:]
df2.head()

In [ ]:
# select first two and last two columns. 
# you can type out each individual column index. 
# But if for example you were using a large dataframe with 100s of columns and you wanted the last n and first n columns
# It can be time consuming to type this out. you can combine both dataframes instead. 

df3 = pd.concat([df.iloc[:,0:2],df.iloc[:,5:]], axis = 1)
df3

In [ ]:
# select 2, 4, 6 columns
df2 = df.iloc[:,[2,4,6]]
df2.head()

identify column index number

In [ ]:
df = trade.copy()
df.columns.get_loc("country_name")

In [ ]:
col_pos = df.columns.get_loc("country_name")
df.iloc[:,col_pos:].head(2) # select column from country_name position to end of df

In [ ]:
df.iloc[:,:col_pos].head(2) # select start of df to index position:

In [ ]:
# this didn't capture country_name - we cna add a +1
df.iloc[:,:col_pos+1].head(2)

#### drop columns

In [ ]:
# remove columns
trade2 = trade.drop(["year","flow","commodity_code"], 1) # index 1 reference columns to remove from df
trade2.dtypes

In [ ]:
trade2 = trade.drop(cols,1)
trade2.dtypes

select columns using column indexes (numbers): tbc:

### 1.a select columns using string patterns

The tariff data uploaded is a good df for this example as it has alot of strings with patterns which can be used for tidy selecitons

In [ ]:
tariff.dtypes

In [ ]:
prefCol = tariff.columns[tariff.columns.str.contains(pat = 'pref')]
prefCol2 = [col for col in tariff.columns if 'pref' in col]

In [ ]:
print(prefCol,prefCol2)

Note difference between output types: one is an indexed array. 

In [ ]:
mfnCol = [col for col in tariff.columns if 'mfn' in col]

In [ ]:
codeCol = [col for col in tariff.columns if 'commodity' in col]

In [ ]:
colNames = [codeCol,mfnCol,prefCol2]
print(colNames)

In [ ]:
#tariff2 = tariff[colNames]
#tariff2.dtypes
# for error fix use:
#colNames = np.concatenate((codeCol,prefCol, mfnCol))

**NOTE the error.** Three list arrays have been combined together which then can't be used in this way to filter a pandas df. 

You can use numpy arrays for the column filters to select the data by using np.concatonate

In [ ]:
prefCol = [col for col in tariff.columns if 'pref' in col]
mfnCol = [col for col in tariff.columns if 'mfn' in col]
codeCol = [col for col in tariff.columns if 'commodity' in col]
colNames2 =  np.concatenate((codeCol,prefCol, mfnCol))
tariff2 = tariff[colNames2]
tariff2.head()

### 1b. select columns with numerical values and combination of string patterns

Select columns which contain numerical values and where numerical values end the column string

i.e. preferntial. + 2021, 2022 etc...

```python
tariff2=tariff[["commodity_code","preferential_applied_duty_rate_2021,
                "preferential_applied_duty_rate_2022",
                "preferential_applied_duty_rate_2023","
                "preferential_applied_duty_rate_2024"]]
```

If there were even more columns to manually type everything out is tedious and time consuming when it can easily be done using string recognition

In [ ]:
col = np.array(tariff.columns[tariff.columns.str.contains('.*[0-9].*', regex=True)]) # select columns with any muerical value
col

doesnt create what is required - can combine str.contains multiple times:

In [ ]:
# doens't work when trying to extract numerical vlaues at end of string: (anyone know fix?)
col_list = [col for col in tariff.columns if col.endswith('.*[0-9].*')]
col_list

In [ ]:
#alternsative quick way can be a simple pattern within the numerical strings, however, extract unwanted tariff columns:
cl = tariff.columns[tariff.columns.str.contains(pat = '20')]
cl

In [ ]:
col = np.array(tariff.columns[tariff.columns.str.contains('20',regex=True)]) # select columns with any muerical value
col

In [ ]:
#example using startswith and endswith:
col_list = [col for col in tariff.columns if (col.startswith('pref') & col.endswith("2"))]
col_list

In [ ]:
c = np.array(tariff.columns[tariff.columns.str.contains(pat = "pref") & tariff.columns.str.contains('20',regex=True)])
c

In [ ]:
# need to combine commoidty code with c in np.array
cd = ["commodity_code"]
c2 = np.concatenate((cd,c))
tariff[c2].head()

In [ ]:
# full solution:
c = np.array(tariff.columns[tariff.columns.str.contains(pat = "pref") & tariff.columns.str.contains('20',regex=True)])
cd = ["commodity_code"]
c2 = np.concatenate((cd,c))
tariff2 = tariff[c2]
tariff2.head()

****

### 1c. Relocate columns:

I am currnelty unaware of a single line function which acheives this like relocate in tidyverse. However it takes a few lines having specified the columns wanting to be relocated within the df.

Example: trade data set - move flow column next to trade value

In [ ]:
trade2 = trade.copy()

In [ ]:
# name column(s) to be moved:
col = trade2["flow"]
# drop column in df
trade2.drop(labels=["flow"], axis = 1, inplace = True)
# insert column back in and select position. Value gbp is column 5(4 when index starts at 0). 
trade2.insert(4,"flow",col)
trade2.head()

In [ ]:
# Can easily move multiple columns using same method:
cols = trade2[["country_name","country_code"]]
col1 = trade2["country_name"]
col2 = trade2["country_code"]
trade2.drop(cols, axis = 1, inplace = True)
# insert column back in and select position. Value gbp is column 5(4 when index starts at 0). 
trade2.insert(1,"country_name",col1)
trade2.insert(1,"country_code",col2)
trade2.head()

If you want to move a larger selection of columns the above method isn't the most helpful. You can more easily specific the seleciton naming the order of columns (similar to select in tidyverse):

In [ ]:
trade2 = trade[["year","country_code","country_name","flow","commodity_code","value_gbp","suppression_notes"]]
trade2.head()

However if you have alot more columns this is also not particularly helpful if you want to decrease time writing out column names..

In [ ]:
#example df:
    
prefCol = [col for col in tariff.columns if 'pref' in col]
mfnCol = [col for col in tariff.columns if 'mfn' in col]
codeCol = [col for col in tariff.columns if 'commodity' in col]
tariffCol = [col for col in tariff.columns if 'tariff' in col]
colNames2 =  np.concatenate((codeCol,prefCol, mfnCol,tariffCol))
tariff2 = tariff[colNames2]
tariff2.dtypes

There are alot of pattenr recogmition strings within this dataframe. However i am approaching this as if there weren't and we wanted to relocate multiple columns ot select positions within a df.

In [ ]:
tariff2 = tariff.copy()

In [ ]:
# relocate MFN columns to front of data frame (method is useful when moving numerous columns to new position)
cols_to_move = ["mfn_applied_duty_rate","mfn_applied_rate_ukgt"]
#col_index = ["commo
tariff3 = tariff2[cols_to_move + [ col for col in tariff2.columns if col not in cols_to_move ]]
tariff3.head(3)

In [ ]:
tariff2.dtypes

In [ ]:
# move pref columns to end of df
cols_to_move = [col for col in tariff.columns if 'pref' in col]
tariff3 = tariff2[[ col for col in tariff2.columns if col not in cols_to_move ]+cols_to_move]
tariff3.dtypes

### **Still looking for solution to move selected columns to arbitary postion in df, i,e, relocate pref columns after "in_quota_tariff_line_code" for example**

****

## 2. Create new columns

creating columns is simple in Python. 

#### Basics

In [ ]:
trade2 = trade.copy()

In [ ]:
trade2["new_col"] = 10
trade2.head(3)

In [ ]:
#convert gbp values:
usd = 0.8
eur = 0.9
trade2["value_usd"] = trade2["value_gbp"]*usd
trade2["value_eur"] = trade2["value_gbp"]*eur
trade2.head(3)

In [ ]:
# add columns together
trade2["new_col"] = trade2["value_gbp"]+trade2["value_usd"]+trade2["value_eur"]
trade2["new_col2"] = trade2["value_gbp"]/100
trade2.head()

In [ ]:
# summarise column values easily:
trade2.sum()

In [ ]:
total_value_gbp = trade["value_gbp"].sum()
total_value_gbp

In [ ]:
# count total number of NANs. Very useful for a quick check.
trade.isnull().sum()

### **Sum across rows:**

In [ ]:
trade2.dtypes

In [ ]:
# example: sum all new column vlaues together
trade2["sum_col"] = trade2["new_col"]+trade2["value_usd"]+trade2["value_eur"]+trade2["new_col2"]

In [ ]:
# alternatively name columns and sum across which is cleaner and less time to type:
sum_cols = ["new_col","value_usd","value_eur","new_col2"]
trade2["sum_col2"] = trade2[sum_cols].sum(axis=1)
trade2.head()

### **Update numerical columns only:**

In [ ]:
# example 1:
#numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
#for c in [c for c in trade.columns if df[c].dtype in numerics]:
#    trade[c] = trade[c]/100

In [ ]:
# smaller one line example:
numeric_df = trade2.apply(lambda x: x/100 if np.issubdtype(x.dtype, np.number) else x)
numeric_df.head()

In [ ]:
# update multiple columns at once:
cols = ["value_eur","value_usd"]
trade2[cols] = trade2[cols]*1000
trade2.head()

### **Combine columns together:**

In [ ]:
trade2 = trade.copy()
trade2["new_col"] = trade2["year"].map(str)+trade2["flow"] # use map(str) as year is numeric column
trade2["new_col2"] = trade2["country_code"]+" - "+trade2["country_name"]
trade2["commoidty_code2"] = "0"+trade2["commodity_code"]
trade2.tail()

### **Conditionally create columns**

There are two useful and simple ways to create and update columns using condiitonal logic

In [ ]:
trade2 = trade.copy()
trade2.head()

**np.where**

In [ ]:
# create column to indicate if value is greater than 100,000
trade2["value_flag"] = np.where(trade2["value_gbp"] > 10000,"Yes","No")
trade2.head()

In [ ]:
# nested np.where statement:
trade["value_flag"] = np.where(trade["value_gbp"] > 100000, "100k",
                               np.where(trade["value_gbp"] > 10000,"10k",
                                        np.where(trade["value_gbp"] > 1000, "1k","<1k"))) # ensure last condition is created
trade

In [ ]:
# can use & or | operaters inside np.where statements
# create flag if country = Taiwan and value is over > 100k
# create flag is value is > 100K or less than 1k

#'' ** ensure both logical conditions are within brackets ()
trade["example_flag"] = np.where((trade["value_gbp"] > 100000) & (trade["country_name"] == "Taiwan"),"Yes","no")
trade["example_flag2"] = np.where((trade["value_gbp"] > 100000) | (trade["value_gbp"] < 1000),"Yes","no")
trade

**np.select**

np.select method when dealing with multiple conditions can be help to write cleaner and more consice code to read and follow. 

This method you specific your conditions and outcomes within an array then define a column using this inputs within np.select

In [ ]:
conditions = [(trade["value_gbp"]>100000), (trade["value_gbp"] >10000), (trade["value_gbp"] >1000)]
choices = ["100k","10k","1k"]
trade['value_flag2'] = np.select(conditions, choices, default="<1k") # chnage default to 0 or any character
trade

****

## 3. Filtering

Filter trade data for simple conditions like year or country name

In [ ]:
# filter for country name = United States
df = trade.copy()
df = df.loc[df["country_name"] == "United States"]
df

In [ ]:
# you don't have ot use loc but I have grown acustomed to this method. 
df = trade.copy()
df = df[df["country_name"] == "United States"]
df.head()

In [ ]:
# filter using opoerators:
# filter for United States, year and flow:
df = trade.copy()
df2 = df.loc[(df["country_name"]=="United States") & (df["year"] == 2020) & (df["flow"] == "Imports")]
df2.head()

In [ ]:
# filter if value of trade is > 10000 or less than < 1000
df2 = df.loc[(df["value_gbp"] > 10000) | (df["value_gbp"] < 1000)]

**Important:** when creating multiple conditions ensure they are within brackets ()

****

### Filter NAs

The UK TRQ data set has multiple NAs throughout which will be a useful dataset to demonstrate

In [ ]:
# utilise .info() for quick overview of Non-Null counts
uk_trqs.info()

In [ ]:
# alternatively - quick simple sum of null values
uk_trqs.isnull().sum()

In [ ]:
# filter df for NAs in geographical areas column
na_df = uk_trqs[uk_trqs['quota__geographical_areas'].isnull()]
na_df.head(2)

In [ ]:
# filter df for not NAs in geographical areas column
not_na_df = uk_trqs[~(uk_trqs['quota__geographical_areas'].isnull())]
not_na_df.head(2)

**Drop NA columns**

In [ ]:
# drop any columns which has an NA value in:
drop_na = uk_trqs.drop(uk_trqs.columns[uk_trqs.isna().sum()>len(uk_trqs.columns)],axis = 1)
drop_na.info()

In [ ]:
# via using a list:
na_cols = uk_trqs.columns[uk_trqs.isna().any()].tolist() # cretae list of columns with NAs in. 
uk_trqs2 = uk_trqs[[col for col in uk_trqs.columns if col not in na_cols]]
uk_trqs2.info()

In [ ]:
# drop columns which only contain NAs i.e. (quota__monetary_unit)
drop_na_cols = uk_trqs.dropna(axis=1, how='all') 
drop_na_cols.info()

****

### Filter by arrays

Rather than using multiple OR opoerators you can use simple arrays to filter your data frame.

Example - filter the trade data for Thailand, Taiwan and United States. 

In [ ]:
df = trade.copy()
df2 = df.loc[(df["country_name"] == "Taiwan") | (df["country_name"] == "Thailand") | (df["country_name"] == "United States")]
pd.unique(df2["country_name"])

In [ ]:
country_array = ["Taiwan","Thailand","United States"]
df2 = df[df["country_name"].isin(country_array)]
pd.unique(df2["country_name"])

This method is storngly preferable whne working with far greater numbers of values to filter by

In [ ]:
code_array = ["01012100","01062000","02031913","02031990","94036090"]
df2 = df[df["commodity_code"].isin(code_array)]
print(pd.unique(df2["commodity_code"]),df2.shape)

In [ ]:
# not in:
code_array = ["01012100","01062000","02031913","02031990","94036090"]
df2 = df[~(df["commodity_code"].isin(code_array))]
print(pd.unique(df2["commodity_code"]),df2.shape)

In [ ]:
# using other column df:
df = trade.head(20)
df2 = trade.head(40)

In [ ]:
# 20 unique codes:
code_filt = pd.unique(df["commodity_code"])
code_filt.shape

In [ ]:
print(pd.unique(df2["commodity_code"]).shape)
# 40 unique codes:

In [ ]:
# filter df2 using df will result in 20 codes:
df3 = df2[df2["commodity_code"].isin(code_filt)]
df3.shape

****

### Filter across columns

filter across columns if value exists, i.e. any vlaue column contains "0". tbc.

In [ ]:
df = tariff.copy()
df.head()

In [ ]:
df.shape

In [ ]:
# filter any column in df which contains text string:
df2 = df[df.stack().str.contains('10%').any(level=0)]
#df2 = df[df.stack().str.contains('7%').any(level=0)]
#df2 = df[df.stack().str.contains('Eggs').any(level=0)]
df2

Alternatively use applymap with la,bda x and any: (will work with non-text strings)

In [ ]:
df2 = df[df.applymap(lambda x: x == "10%").any(1)]
df2

Filter select columns rather than all df columns:

In [ ]:
# select columns to filter across. can use iloc, name columns or use string recognition:
#1
col_names = df.iloc[:,[5,6,7,8]]
col_names = col_names.columns
col_names
#2
col_names = ["preferential_applied_duty_rate_2021","preferential_applied_duty_rate_2022","preferential_applied_duty_rate_2023","preferential_applied_duty_rate_2024"]
#3
col_names = np.array(tariff.columns[tariff.columns.str.contains(pat = "pref") & tariff.columns.str.contains('20',regex=True)])
col_names

In [ ]:
# filter across rows hwere pref tariff == x
df2 = df[(df[col_names] == "2%").any(1)]
df2

****

## 4. Aggregations

Data transformations grouping and aggregating data is one of the most common practices I and our department does. We extract and clean large maounts of data aggregating it to more actionable outputs with teams. Groupby is essential and straight forward for aggregations. 

I will be demonstrating aggregation using the trade data set which is a very rich and useful dataset as there are multiple ways to group and summarise the data which would be useful for people. 

In [ ]:
df = trade.copy()
df["value_gbp2"] = df["value_gbp"]*10

In [ ]:
# group by year - sum total value. Notice difference when keeping index as false:
df_agg  = df.groupby(["year"])["value_gbp"].sum()
df_agg

In [ ]:
df_agg2 = df.groupby(["year"], as_index = False)["value_gbp"].sum()
df_agg2

In [ ]:
# group by using count and mean
df_agg = df.groupby(["year"])["value_gbp"].count()
# df_agg = groupby(["year"])["value_gbp"].mean()
df_agg

In [ ]:
# multiple calculations of same column:
df_agg = df.groupby(["year"]).agg({"value_gbp": ["sum","mean","count","max","min"]})
df_agg

In [ ]:
# Multiple grouping for year and country
df_agg = df.groupby(["year","flow"]).agg({"value_gbp": ["sum","mean","count","max","min"]})
df_agg

In [ ]:
# seperate aggregate calculations:
df_agg = df.groupby(["year","flow"], as_index = False).agg({"value_gbp":"sum","value_gbp2":"mean"})
df_agg

****

### Conditional aggreations (similar to sumif in excel)

In [ ]:
# Calculate total trade values for each year and trade flow for America:

In [ ]:
df_agg = df.groupby(["year","flow"]).apply(lambda x: x[x['country_name'] == 'United States']['value_gbp'].sum())
df_agg

In [ ]:
# alternative way using assign and numpy:
df_agg = df.assign(
    us1 = np.where(df["country_name"]=="United States",df.value_gbp,0),
    us2 = np.where(df["country_name"]=="United States",df.value_gbp2,0)
   ).groupby("year").agg({"us1":"sum","us2":"mean"})

df_agg

This method is handy if you wanted to conditionally aggregate specific countries into a wider dataframe. For Example - what are the year trade values of Taiwan and Thailand:

In [ ]:
df_agg = df.assign(
    thailand = np.where(df["country_name"]=="Thailand",df.value_gbp,0),
    taiwan = np.where(df["country_name"]=="Taiwan",df.value_gbp,0)
   ).groupby(["year","flow"],as_index=False).agg({"thailand":"sum","taiwan":"sum"})

df_agg

### Grouped calculations

 Rather than aggregated data using group by - we can grouped dataframes and calculate columns form this utilising groupby and apply. 

For example you may with to see the largest value as a proportion of the individual group - rather than the entire dataset. 

Taking the trade data set we can aggregate total trade for each country across each year. We then want to calulcate how much % each country represents for total trade across each year. 

In [11]:
df = trade.copy()
df_agg = df.groupby(["year","country_code","country_name"], as_index = False).agg({"value_gbp":"sum"})
df_agg["prop"] = df_agg["value_gbp"] / sum(df_agg["value_gbp"]) 
df_agg.head()
# trying this results in each countries value being divided by the entire total sum of the value column - rather than the sum of the group (year/country)

,year,country_code,country_name,value_gbp,prop
0,2019,TN,Tunisia,6822944,0.00006
1,2019,TW,Taiwan,361582827,0.00331
2,2019,US,United States,7982756283,0.07297
3,2019,UY,Uruguay,63050111,0.00058
4,2019,UZ,Uzbekistan,4721840369,0.04316


In [14]:
df_agg["perc"] = df_agg.groupby(["year"])["value_gbp"].apply(lambda x: x/x.sum())
df_agg
# compare the proportion vs percentage column
# the perc. correctly calculates the proportion of the value related to the group specified (i.e year). 

,year,country_code,country_name,value_gbp,prop,perc
0,2019,TN,Tunisia,6822944,0.00006,0.00051
1,2019,TW,Taiwan,361582827,0.00331,0.02695
2,2019,US,United States,7982756283,0.07297,0.59488
3,2019,UY,Uruguay,63050111,0.00058,0.00470
4,2019,UZ,Uzbekistan,4721840369,0.04316,0.35187
5,2019,VA,Vatican City State,251330,0.00000,0.00002
6,2019,VE,"Venezuela, Bolivarian Republic of",34499624,0.00032,0.00257
7,2019,VI,United States Virgin Islands,462392,0.00000,0.00003
8,2019,VN,Vietnam,242112223,0.00221,0.01804
9,2019,VU,Vanuatu,89558,0.00000,0.00001


****

### Unnest equivilent 

Un-concatonate a cell broken up by delimiter into new seperate rows inside a df. 

upload uk_trq data with commodity codes concatoneted together in one column seperated by a delimiter. 

In [ ]:
uk_trqs = pd.read_csv("data/uk_trqs.csv",dtype={'quota__order_number': str}) # upload xlsxl
uk_trqs.head()

In [ ]:
# select columns to groupby and to unconcatonate. In this instance we have a quota level daa frame. So we select the quota order number and commodity codes. 
df = uk_trqs[["quota__order_number","quota__commodities"]]
df.info()

In [ ]:
# NOTE the below unnest steps won't work if NaN present in data:
# remove NaN values. 
df = df.loc[~df["quota__commodities"].isnull()]

In [ ]:
# following steps to split out each cell within delimiters and create new row:
new_df = pd.DataFrame(df.quota__commodities.str.split('|').tolist(), index=df.quota__order_number).stack()
new_df = new_df.reset_index([0, 'quota__order_number'])
new_df.columns = ['quota__order_number', 'quota__commodities']
new_df['quota__order_number'] = new_df[ 'quota__order_number'].str.strip() # remove whitespace
new_df

****

## 5. Joins

In [ ]:
# simple left join using dfs with unique rows with simple one to one relationship:

In [ ]:
df = trade.groupby("country_name").mean()
df2 = trade.groupby("country_name").sum()

In [ ]:
# two dataframes with same index, can join using index
df3 = pd.merge(df,df2, left_index = True, right_index = True)
df3.head()

In [ ]:
# you can use concat with using outer join by default - using axis = 1. axis = 0 combined rows. 
df3 = pd.concat([df2,df2], axis = 1)
df3

In [ ]:
# you can combine multiple dfs together using concat:
df4 = pd.concat([df,df2,df3], axis = 1) # again note - to bind together rows chnage axis to 0. 
df4.head()

In [ ]:
# merge not using index:
df = trade.groupby("country_name", as_index = False).mean()
df2 = trade.groupby("country_name", as_index = False).sum()
df3 = pd.merge(df,df2, on = "country_name", how = "left")
df3

**NOTE:** when different column index names, use "left_on" and "right_on"

```python

By Default:

join  is a column-wise left join
pd.merge is a column-wise inner join
pd.concat  is a row-wise outer join

```

****

## 6. Other operators

#### remove duplicates

In [ ]:
df = trade.head(20).copy()
df2 = trade.head(20).copy()
df3 = pd.concat((df,df2))
# df3 dup;icated dataframe
df3.shape

In [ ]:
# removing all rows which are duplicates will result in 20 row df:
df_dup_remove = df3.drop_duplicates() # drop_duplicates() will remove where rows are the same across all cells. 
df_dup_remove.shape

In [ ]:
# remove duplicates based on columns. For example remvoe duplicates for "year" which will result in two rows.
df_dup_remove = df3.drop_duplicates(subset = ["year"])
df_dup_remove

In [ ]:
df = trade.copy()
df_dup_remove = df.drop_duplicates(subset = ["flow","year"]) 
df_dup_remove

#### sort data

In [ ]:
# sort data based on values:
df = trade.copy()
df2 = df.sort_values("value_gbp") # ascending = False for reverse order
df2

In [ ]:
df2 = df.sort_values(by=["year","flow","value_gbp"])
df2

In [ ]:
# sort dataframe by highest traded commodity for each country for each year. 
df_sorted = df.sort_values(by=["country_name","year","value_gbp"], ascending = False)
df_sorted
# note with combining this while the highest traded codes are ordered first, the alphabetical order has revered. We want the df in alphabetical order. 

In [ ]:
# sort by grouping and value by ascending - then take top two values from each country:
df_sorted2 = df.sort_values(by=["country_name","year","value_gbp"], ascending = False).groupby('country_name').head(2)
df_sorted2.head(10)

In [ ]:
# order by grouping. If all 3 columns are within sort values and by ascedning
# all columns are ordered by ascending when we only want the value column to be highest first. 

df_sorted = df.groupby(["country_name","year"], as_index = False).apply(lambda x: x.sort_values(["value_gbp"], ascending = False))
df_sorted

## Glossary

### 0. Set up

```python

# simple uploads

import pandas as pd
import numpy as np

pd.read_excel('filepath') 
pd.read_csv('filepath')

pd.read_excel('filepath', dtype={'column': str}) # convert "column" to string when uploading data
pd.read_excel("filepath",dtype=str) # convert all columns to string

```

```python
# simple data exploration

df.dtypes # column types
df.info # dataframe info, covers dataframe types, NaNs. 
df.shape # shape of df, i.e. number of rows, columns. 
df.describe() # summarise numerical values

df.head() 
df.tail()

```

```python
# clean column names

df.columns = df.columns.str.lower().str.replace(" ","_")

```

### 1. Select columns

```python

# basic selection 

df[["col1","col2","col3"]] # ensure double square brackets [[]]

# selection using array

array = ["col1","col2","col3"]
df[array]

# drop columns

df.drop(["col2","col3"], 1)

```

#### 1a select using pattern recognition

```python

pattern_col = [col for col in df.columns if 'pattern' in col]
pattern_col2 = [col for col in df.columns if 'pattern2' in col]

# combine using np.concatonate to filter df:

colNames =  np.concatenate((pattern_col, pattern_col2))
new_df = df[colNames]

```

#### 1b select columns with numerical values

```python

cols = np.array(df.columns[df.columns.str.contains('.*[0-9].*', regex=True)]) # select columns with any muerical value

# pattern using endswith and startswith
cols = [col for col in df.columns if col.endswith('.*[0-9].*')]
cols = [col for col in df.columns if col.startswith('.*[0-9].*')]


# combine numerical pattenr recongition with string

col_list = [col for col in df.columns if (col.startswith('pattern') & col.endswith("2"))]

col_list = np.array(df.columns[df.columns.str.contains(pat = "pref") & df.columns.str.contains('20',regex=True)])

```

#### 1c relocate columns

```python

col = df["col1"]
# drop column in df
df.drop(labels=["col1], axis = 1, inplace = True)
df.insert(3,"col1",col) # 3 is column position (chnage to index number you want)

# move multiple columns to start or end of df:
                
cols_to_move = ["col1","col2","col3"]
               
df2 = df[cols_to_move + [ col for col in df.columns if col not in cols_to_move ]]
df2 = df[[ col for col in df.columns if col not in cols_to_move ]+cols_to_move]
                
              
```

### 2. Create new columns

##### simple creation of columns

```python
# new columns are simple to create:

df["new_col"] = 10 # value in all cells
df["new_col"] = df["value_col"]*10
df["new_col"] = df["value_col1"] + df["value_col2"]

# sum across rows:

sum_cols = ["col1","col2","col3","col4"]
df["sum_col"] = df[sum_cols].sum(axis=1)

```

##### update numerical columns:

```python
# labda defined function (example columns dividing by 100):

df = df.apply(lambda x: x/100 if np.issubdtype(x.dtype, np.number) else x)

```

##### update multiple cdefined columns at once:

```python
cols = ["col1","col2","col3]
df[cols] = df[cols]*1000

```

##### combine columns together

```python
# equivalent to using paste in R - concatonate columns together

# use "+"

df["new_col"] = df["value_col"].map(str)+df["col2"] # use map(str) as year is numeric column
df["new_col"] = df["col1"]+" - "+df["col2"] # create string combining two columns seperating out "-"
df["new_col"] = "0"+df["col1"] # combine simple string with column

```

#### conditionally create columns

##### np.where

```python
# equilvaent to R - using mutate combine with ifelse. Somethign used commonly. 


# np.where

df["new_col"] = np.where(df["value_col"] > 10000,"Yes","No")

# nested np.where statement:

df["value_flag"] = np.where(df["value_col"] > 100000, "100k",
                               np.where(df["value_col"] > 10000,"10k",
                                        np.where(df["value_col"] > 1000, "1k","<1k")))

# using logicial operaters: 
# ensure each condition is inside a bracket ()
df["example_flag"] = np.where((df["value_col"] > 100000) & (df["col"] == "Taiwan"),"Yes","no")
df["example_flag2"] = np.where((df["value_col"] > 100000) | (df["value_col"] < 1000),"Yes","no")

```

##### np.select

np.select is very useful for writing more concise and clean code when multiple conditions

```python
conditions = [(df["value_col"]>100000), (df["value_col"] >10000), (df["value_col"] >1000)]
choices = ["100k","10k","1k"]
df['value_col'] = np.select(conditions, choices, default="<1k") # chnage default to 0 or any character

```

### 3. Filtering

##### basic filtering



```python

there are multiple ways to filter a dataframe. The only two I use for simple filtering are:
    
df = df[df["col"] == "condition"]
df = df.loc[df["col"] == "condition]
            

```

```python 

filter using operators (ensure use of brackets () )
    
df = df.loc[(df["col1"] == "condition") & (df["col2"] == "condition2")]

df = df.loc[(df["val1"] > 100) | (df["val2"] < 10)]
```

##### filter NAs

```python

# filter where col is NaN
na_df = df[df["col"].isnull()]

# filter where col is NOT NaN

not_na_df = df[~(df["col"].isnull())]
```

##### Drop columns with NaN

```python
drop_na = df.drop(df.columns[df.isna().sum()>len(df.columns)],axis = 1)

# OR

na_cols = df.columns[df.isna().any()].tolist() # cretae list of columns with NAs in. 
drop_na = df[[col for col in df.columns if col not in na_cols]]


# drop columns which only contain NAs 
drop_na_cols = df.dropna(axis=1, how='all') 

```

##### filter by arrays

```python

array = ["value1","value2","value3"]
df2 = df[df["value_col"].isin(array)]

# not in
array = ["value1","value2","value3"]
df2 = df[~(df["value_col"].isin(array))]

```

##### filter across columns

```python
# filter any columns which contains string:

df = df[df.stack().str.contains('string').any(level=0)]

# OR

df = df[df.applymap(lambda x: x == "string").any(1)]

# filter across selected columns:

columns_to_filt = ["col1", "col2", "col3", "col4"]

df2 = df[(df[columns_to_filt] == "condition").any(1)] 

```

### 4. Aggregations

##### basic aggregations using groupby

```python
# basic and quick aggregation:

df.groupby(["col_agg"])["col_value"].sum # (.count, .mean etc)
df.groupby(["col_agg"], as_index = False)["col_value"].sum # use as_index = False to remove index and have as a column

# multiple calculations one one value

df.groupby(["col_agg"]).agg({"col_value": ["sum","mean","count","max","min"]})

# multiple conditions within aggregation:

df.groupby(["col_agg1","col_agg2","col_agg3"]).agg({"col_value": "sum"})

# seperate aggregate calculations:
df.groupby(["col_agg1","col_agg2"], as_index = False).agg({"col_value1":"sum","col_value2":"mean"})

```

##### grouped calculations

```python
# ensure first grouping
df_agg = df.groupby(["col1","col2"].agg({"value":"sum"})
                    
# grouped calculation
df_agg["perc"] = df_agg.groupby(["col1"])["value"].apply(lambda x: x/x.sum())
```

##### conditional aggregations

```python

# equivalent to sumif
df.groupby(["col_agg").apply(lambda x: x[x['col'] == 'condition']['col_value'].sum())
            
# alternative way using assign and numpy:
            
df_agg = df.assign(
    val1 = np.where(df["col"]=="condition",df.col_value1,0),
    val2 = np.where(df["col"]=="condition",df.col_value2,0)
   ).groupby("col_agg").agg({"val1":"sum","val2":"mean"})


```

### 5. Joins

```python

df3 = pd.merge(df,df2, left_index = True, right_index = True)

# merge using defined "how"
df3 = pd.merge(df,df2,on = "joinID", how = "left") # can be placed by right, inner etc. 

# concat (default outer join if index is 1, 0 for row bind

df4 = pd.concat([df,df2,df3], axis = 1) 
df4 = pd.concat([df,df2,df3], axis = 0) 


By Default:

join  is a column-wise left join
pd.merge is a column-wise inner join
pd.concat  is a row-wise outer join

```

### 6. Other operators 

#### remove duplicates

```python
df.drop_duplicates()
df.drop_duplicates(subset=["col1","col2"])

```

#### sort values

```python

df.sort_values(by=["col1"])
df.sort_values(by=["col1"], ascending = False) # highest value first
df.sort_values(by=["col1","col2","col3"]) # multiple grouping 

# sort grouped data
df.groupby(["group1","group2"]).apply(lambda x: x.sort_values(["sort_column"], ascending = False))

```

End.